In [1]:
#importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from linear_regression import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from create_final_df import CreateDataframe
from polynomial_regression import PolynomialRegression
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
lr = LinearRegression()
df = CreateDataframe()
confirmed_df = df.get_final_df('Confirmed')
deaths_df = df.get_final_df('Deaths')
recovered_df = df.get_final_df('Recovered')
confirmed_df.head(110)

,Days,Date,Georgia,North Carolina,Wyoming,New Hampshire,Wisconsin,Minnesota,Maryland,Pennsylvania,...,Idaho,Oregon,Iowa,Vermont,West Virginia,Kansas,New Jersey,Washington,Missouri,Rhode Island
0,1,04-12,12452,4570,270,929,3341,1621,8225,22938,...,1407,1527,1587,727,593,1344,61850,10609,4269,2665
1,2,04-13,13315,4886,275,985,3428,1621,8936,24292,...,1426,1584,1710,748,611,1390,64584,10635,4515,2665
2,3,04-14,14578,5113,282,922,3555,1695,9472,25465,...,1464,1633,1899,752,640,1441,68824,10799,4746,3251
3,4,04-15,14987,5340,287,1139,3721,1809,10032,26753,...,1473,1663,1995,759,702,1504,71030,10942,4791,3251
4,5,04-16,15669,5639,296,1139,3875,1809,10784,28258,...,1587,1736,2141,774,728,1615,75317,11057,5174,3529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,106,07-26,167953,112937,2475,6436,48827,51153,83748,111745,...,18178,16758,42357,1400,5889,25290,179363,52635,42160,18224
106,107,07-27,170843,114916,2520,6441,49417,51803,84876,112995,...,18692,17088,42696,1402,5999,25804,179812,53321,43489,18515
107,108,07-28,175052,116700,2589,6500,50179,52281,85524,114083,...,19220,17408,42928,1405,6120,26058,180295,54205,45181,18725
108,109,07-29,178323,118387,2628,6513,51049,52947,86285,114939,...,19674,17721,43623,1406,6277,26591,180600,54985,46942,18800


In [3]:
#Create a dictionary of DF
confirmed_states_df = {}
training_days = np.array(confirmed_df["Days"]).reshape(-1, 1)
# print(training_days)
for column in confirmed_df: 
    if column != 'Days' and column != 'Date':
       confirmed_states_df[column] = confirmed_df[column]    
# # for state in confirmed_states_df:
# print(confirmed_states_df['Georgia'].values)
# scalar = StandardScaler()
# x = scalar.fit_transform(confirmed_states_df[state].values)
# print(x)

In [4]:
# # creating list to store PR(degree=2) objects of states     
# confirmed_states_model_list = [] 
# for state_name, state_df in confirmed_states_df.items():
# #     print(state_df[:30])
# #     print(state_name)
# #     print(state_df)
#     pr_degree_2 = PolynomialRegression(3, state_name)
#     pr_degree_2.train(training_days, state_df[:30])
#     confirmed_states_model_list.append(pr_degree_2)

In [5]:
# #PR Degree 2 predicitions, store prediciton fo each state in a dict
# confirmed_states_pred_dict = {} 
# period = 100
# # print(period)
# upcoming_days = 10
# days = np.array(range(period, period + upcoming_days)).reshape(-1, 1)
# for model in confirmed_states_model_list:
#     confirmed_states_pred_dict[model.state_name] = model.get_predictions(days)

In [6]:
# #Just testing California
# print("The prediction in Georgia July 21st till July 30st are: ")
# # print(np.reshape(confirmed_df['Georgia'][30:40],(10,1)))
# # print(confirmed_df['Georgia'][30:40].shape)
# ground_truth = np.array(confirmed_df["Georgia"][100:110,]).reshape(-1,)
# prediction = confirmed_states_pred_dict['Georgia']

# for i in range(len(prediction)):
#     print('prediction for date: July ' + str(i + 21), prediction[i])
# print(ground_truth)
# # print(lr.compute_mse(prediction,ground_truth))
# print('mean squared error:', metrics.mean_squared_error(ground_truth, prediction, squared=False))


In [7]:
# def crossValidate(model, X, y, ntrials=100, test_size=0.2, degree=2):
    
#     sss = StratifiedShuffleSplit(n_splits=ntrials, train_size=(1-test_size), test_size=test_size, random_state=0)
#     average_mse = 0.0
    
#     print(y.shape)
    
#     for train_i, test_i in sss.split(X, y):
#         X_train, X_test = X[train_i], X[test_i]
#         y_train, y_test = y[train_i], y[test_i]
        
#         poly = PolynomialFeatures(degree=degree)
#         X_train_transform = poly.fit_transform(X_train)
        
#         model.fit(X_train_transform, y_train)
        
#         X_test_transform = poly.fit_transform(X_test)
#         y_pred = model.predict(X_test_transform)
#         average_mse += metrics.means_squared_error(y_test, y_pred, squared=False)
        
#     average_mse /= ntrials
    
#     return average_mse



# print(sorted(metrics.SCORERS.keys()))
lr = LinearRegression()
    
poly = PolynomialFeatures(degree=4)
X_transform = poly.fit_transform(np.array([[num] for num in range(0, 142)]))

lr.fit(X_transform, np.array(confirmed_df['Georgia']).reshape(-1,))
y_pred = lr.predict(X_transform)
print(y_pred - np.array(confirmed_df['Georgia']).reshape(-1,))

validation_errors = []
for i in range(2, 20):
    lr = LinearRegression()
    
    poly = PolynomialFeatures(degree=i)
    X_transform = poly.fit_transform(np.array([[num] for num in range(0, 142)]))
    validation_errors += [math.sqrt(- np.average(cross_val_score(lr, X_transform, y=np.array(confirmed_df['Georgia']).reshape(-1,), scoring='neg_mean_squared_error', cv=10)))]
    
print(validation_errors)

# for i in range(2, 20):
#     print(np.array(confirmed_df['Georgia']).reshape(-1,).shape)
#     print(np.array([[num] for num in range(0, 142)]).shape)
#     error = crossValidate(lr, np.array([[num] for num in range(0, 142)]), np.array(confirmed_df['Georgia']).reshape(-1,), degree=i)
#     print('average_mse for i = %d: %.3f' % (i, error))
    
    

[-7742.37572008 -6149.85939765 -5106.62167574 -3352.61092338
 -2008.86260197 -1639.49926538  -345.73055989   671.14677581
  1099.7489106   2051.60492093  2041.15679083  2596.75941192
  3120.68058338  3434.10101197  4132.11431204  4200.7270055
  4392.85852184  4284.34119815  4476.92027906  3875.2539168
  3602.91317119  3789.38200959  3479.05730696  3367.24884584
  3021.17931634  2512.98431615  2288.71235053  2196.32483232
  1498.69608195  1354.61332741   697.77670427   450.79925569
   151.20693239  -395.56140732  -586.15399757  -699.30616489
 -1141.84032824 -1441.66599902 -2101.77978103 -2661.26537051
 -3158.29355611 -3574.12221892 -3865.09633242 -3966.64796256
 -4123.29626767 -4319.64749854 -4454.39499834 -4539.3192027
 -4420.28763965 -4542.25492967 -4728.26278563 -4335.44001285
 -4278.00250905 -4450.25326439 -4369.58236144 -4213.46697521
 -3805.47137313 -3403.24691503 -3098.53205318 -2716.15233228
 -2538.02038944 -2118.1359542  -1849.58584853 -1376.54398679
  -698.27137581   110.88388